# Application of Statistical Testing in Python

While Python is a powerful, general purpose, language that is widely used in data science, R is a domain-specific language designed for statistical analysis. Its libraries are often released by researchers and the documentation and code is accompanied by citations from scientific papers. The application of statistical formulas are much simplier with R language and the outputs tend to be more informative than their counterparts in Python (although there are packages in Python that can provide a similar framework to R, i.e. statsmodels). 

The purpose of this project is to demonstrate through examples how one can apply statistical testing in Python using only SciPy.stats to achieve the similar functionality as R. 

### Contents

<ul>
  <li>1. Parametric (Normal Distribution) Testing:   
    <ul>
      <li>1.1 Comparing Two Groups:
        <ul>
          <li>1.1.1 Unpaired: sample t-tests</li>
          <li>1.1.2 Paired: paired t-test</li>
          <li>1.1.3 Chi-square test for variance</li>
        </ul>
      </li>
      <li>1.2 Comparing More than Two Groups:
        <ul>
          <li>1.2.1 Sample design: one way ANOVA</li>
          <li>1.2.2 Block design: two way ANOVA</li>
        </ul>
      </li>
     </ul>
    </li>
    <li>2. Non-Parametric (Non-Normal Distribution) Testing:   
      <ul>
        <li>2.1 Comparing Two Groups:
           <ul>
             <li>2.1.1 Unpaired: Wilcoxon rank-sum test</li>
             <li>2.1.2 Paired: Wilcoxon signed-rank test </li>
           </ul>
        </li>
        <li>2.2 Comparing More than Two Groups:
           <ul>
             <li>2.2.1 Sample design: Kruskal-Wallis test</li>
             <li>2.2.2 Block design: Friedman test</li>
           </ul>
        </li>
      </ul>
    </li>
</ul>

# 1. Parametric (Normal Distribution) Testing

Parametric tests makes assumptions of the underyling distribution of the data. These assumptions must be met for the tests to be considered reliable. For example, for a paired t-test for two samples, the test is considered reliable only if both samples follow a normal distribution and have the same variances. 

## 1.1 Comparing Two Groups

In [5]:
import numpy as np
import pandas as pd

from scipy.stats import ttest_ind, ttest_1samp, ttest_rel, chisquare, chi2, t
from math import sqrt

### Unpaired Sample t-tests
    
<b> One sample, two tailed t-test</b> 

A one sample, two tailed t-tests determines whether the sample mean is statistically different from a known or hypothesized population mean. For this scenerio, SciPy's "ttest_1samp" function is used.

Example 1: Test with a 95 percent confidence interval whether the volume of a shipment of lumber is as per usual the 39,000 cubic feet. Suppose the shipment mean is 36,500 cubic feet with a standard deviation of 2,000. So the null hypothesis is Ho: mu = 39000.

In [3]:
data_mean = 36500
data_std = 2000

data = np.random.normal(loc=data_mean, scale=data_std, size=75)
print ('mean: %.5f' % data.mean(), 'std: %.5f' % data.std(ddof=1))
# Note: R calulates the standard deviation with N - 1 as the denominator, and numpy with N. 
# To get identical results, set ddof = 1 ("delta degrees of freedom").

mean: 36507.58517 std: 1987.60671


In [6]:
ttest_1samp(data, 39000)

Ttest_1sampResult(statistic=-10.859766890351752, pvalue=5.662451054667056e-17)

In this above example, the p-value is greater than 0.05, we cannot reject the null hypothesis that the sample mean is equal to the population mean. We can also define a function to output a message advising whether to reject the null hypothesis at a specified confidence level (i.e. 0.1, 0.05, 0.01).

Example 2: The results obtained for an intelligence test in 10 subjects are: 65, 78, 88, 55, 48, 95, 66, 57, 79, 81. Test with a 99% confidence interval that the population which received the same test is equal to 75.

In [7]:
results = np.array([65,78,88,55,48,95,66,57,79,81])
print ('mean: %.5f' % results.mean(), 'std: %.5f' % results.std(ddof=1))

mean: 71.20000 std: 15.34637


In [8]:
def one_sample_two_tailed(data, pop_mean, alpha):
    t, p = ttest_1samp(data, pop_mean)
    print ('t: %.5f' % t)
    print ('p: %.5f' % p)
    if p < alpha:
        print('P-Value is less than alpha: We can reject the null hypothesis')
    if p > alpha:
        print('P-value is greater than alpha: We cannot reject the null hypothesis')

In [9]:
one_sample_two_tailed(results, 75, alpha=0.01)

t: -0.78303
p: 0.45372
P-value is greater than alpha: We cannot reject the null hypothesis


<b> One Sample, one tailed t-test</b> 

A one sample, one tailed t-tests determines whether the sample mean is greater or less than a known or hypothesized population mean. The t.test function in R allows you to specify the upper or lower tail with the "alternaitve" parameter. However, the ttest_1samp has no such functionality. We can define a function with a similar parameter which take in 'less' or 'greater' as the alternative hypothesis. 

Example 1: A bottle filling machine is set to fill bottles with a volume of 500 ml, which the actual volume known to follow a normal distribution. The manufacturer believes the machine is under-filling bottles so a sample of 20 filled bottles is taken and the volume in each was recorded as: 484.11, 459.49, 471.38, 512.01, 494.48, 528.63, 493.64, 485.03, 473.88, 501.59, 502.85, 538.08, 465.68, 495.03, 475.32, 529.41, 518.13, 464.32, 449.08, 489.27.

Use a one-sample t-test to determine whether the bottles are being under filled using a significance level of 0.01.

In [13]:
volume = np.array([484.11, 459.49, 471.38, 512.01, 494.48, 528.63, 493.64, 485.03, 473.88, 501.59, 502.85, 538.08, 465.68, 495.03, 475.32, 529.41, 518.13, 464.32, 449.08, 489.27])
print ('mean: %.5f' % volume.mean(), 'std: %.5f' % volume.std(ddof=1))

mean: 491.57050 std: 24.79368


The below function defines a one sample one tailed test based off the following intuition:

- Suppose the alternative hypothesis is μ > 0. In this case if the sample mean is negative, we cannot reject the null that the true mean is zero in favour of the alternative that it is positive if the data suggests that the mean is negative.
- Furthermore, a positive t-statistic implies that the sample mean is larger than the hypothesized mean.  This would be evidence against the null hypothesis IF (and only if) the alternative was that the true mean is GREATER than the hypothesized value.

- Suppose the alternative hypothesis is μ < 0. If the sample mean is positive, then we cannot reject the null hypothesis that the true mean is zero in favour of the alternative that it is negative. 
- A negative t-statistic implies that the sameple mean is less than the hypothesized mean, which is evidence against the null hypothesis IF (and only if) the alternative was that the true mean is LESS than the hypothesized mean. 

In [14]:
def one_sample_one_tailed(data, pop_mean, alpha, alternative):
    t, p = ttest_1samp(data, pop_mean)
    print('t: %.5f'% t)
    if (t < 0 and alternative == 'less') | (t > 0 and alternative == 'greater'):
        p = p/2
        print('p: %.5f'% p)
    if (t < 0 and alternative == 'greater') | (t > 0 and alternative == 'less'):
        p = 1-(p/2)
        print('p: %.5f'% p)
    if p < alpha:
        print('%.5f'% p, 'is less than', alpha, 'therefore reject null hypothesis')
    if p > alpha:
        print('%.5f'% p, 'is greater than', alpha, 'therefore cannot reject null hypothesis')

In [15]:
one_sample_one_tailed(volume, 500, alpha=0.01, alternative='less')

t: -1.52046
p: 0.07243
0.07243 is greater than 0.01 therefore cannot reject null hypothesis


<b> Independent Two Sample t-test</b> 

The two-sample t-test is used to determine if two samples means are equal. The fucntion "ttest_ind" is used for two sample t-tests. However, unlike R's t.test function, the output from ttest_ind does not provide the confidence intervals of the difference in means. To overcome this, two seperate functions are defined based on pooled or unpooled variances.

Example 1: Assuming that the data in mtcars follows the normal distribution, find the 95% confidence interval estimate of the difference between the mean gas mileage of manual and automatic transmissions.

In [16]:
mtcars = pd.read_csv("https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv")
mpg_auto = mtcars['mpg'][mtcars['am']==0]
mpg_manual = mtcars['mpg'][mtcars['am']==1]

In [17]:
# Calculate t-test for 2 independent samples with unequal variances. 
ttest_ind(mpg_auto, mpg_manual, equal_var= False)

Ttest_indResult(statistic=-3.767123145144923, pvalue=0.0013736383330710345)

From the above output, we can reject the null hypothesis that the two samples are equal. The following function calculates the 95% confidence interval for the difference between two means given unequal variances between the two samples. 

Note that there are two formuals to calculate the CI based on whether the variances are pooled or unpooled. The unpooled method is known as the Welch-Satterthwaite method.

In [18]:
# Function to calculate 95% confidence intervals for unpooled variances 
def confit_welch_2samp(x1, x2):
    diff_mean = x1.mean() - x2.mean()
    N1 = x1.shape[0]
    N2 = x2.shape[0]
    # Degrees of Freedom
    df = (((x1.std()**2 /N1) + (x2.std()**2 /N2))**2) / ((x1.std()**4)/((N1**2)*(N1-1)) + (x2.std()**4)/((N2**2)*(N2-1)))
    # Standard Error
    se = sqrt((x1.std()**2/N1) + (x2.std()**2/N2))
    # T-value
    t_val = t.ppf(0.975, df)
    # Margin of Error
    MoE = t_val * se
    #95% Confidence Interval
    conf_int = diff_mean - MoE, diff_mean + MoE    
    print('95 percent confidence interval:', conf_int)

In [19]:
confit_welch_2samp(mpg_auto, mpg_manual)

95 percent confidence interval: (-11.280194355040031, -3.2096841874700814)


This tells us that with 95% confidence the difference in mean mpg between auto and manual transmission is between -11.28 and -3.21.

Example 2: Consider the weight gain of 19 rats between 28 and 84 days after birth. 12 were fed on a high protein diet and 7 on a low protein diet. Using the following data, test the hypothesis that there is no difference in weight gain between rats raised on a high-protein diet versus a low-protein diet. Use a significance level of α = 0.05 and assume equal variances. 

High protein: 134,146,104,119,124,161,107,83,113,129,97,12  Low protein: 70,118,101,85,107,132,94

In [20]:
high_protein = np.array([134,146,104,119,124,161,107,83,113,129,97,123])
low_protein = np.array([70,118,101,85,107,132,94])
ttest_ind(high_protein, low_protein)

Ttest_indResult(statistic=1.89143639744233, pvalue=0.07573012895667763)

The below formula takes in two vectors and calculates the 95% confidence interval for a difference in means assuming equal variances between both samples. 

In [21]:
# 95% confidence interval for pooled variances 
def confint_2samp(x1, x2):
    diff_mean = x1.mean() - x2.mean()
    N1 = x1.shape[0]
    N2 = x2.shape[0]
    df = N1 + N2 - 2
    #t-statistic 
    t_val = t.ppf(0.975, df)    
    std_N1N2 = sqrt(((N1 - 1 )*(x1.std())**2 + (N2 - 1)*(x2.std())**2) / df)
    # Margin of Error
    MoE = t_val * std_N1N2 * sqrt(1/N1+ 1/N2)   
    #95% Confidence Interval
    confint = diff_mean - MoE, diff_mean + MoE
    print('95 percent confidence interval:', confint)

In [22]:
confint_2samp(high_protein, low_protein)

95 percent confidence interval: (-1.068489658012112, 39.06848965801211)


### Paired Sample t-test 

The paired t-test is used to determine the difference in means between two samples for the same subject, for example the samples are seperated by time. With SciPy, the ttest_rel function is used to conduct the paired t-test. 

Example: In the immer dataset, Y1 represents yield in 1931, Y2 in 1932. Assuming that the data in immer follows the normal distribution, find the 95% confidence interval estimate of the difference between the mean barley yields between years 1931 and 1932.

In [ ]:
# Found on GitHub
immer = pd.read_csv("immer.csv")

In [109]:
# Use ttest_rel for paired samples
ttest_rel(immer['Y1'], immer['Y2'])

Ttest_relResult(statistic=3.3239873042716788, pvalue=0.0024126338636167597)

Due to the low p-value we can reject the null hypothesis that the mean yields in 1931 and 1932 are the same. 

SciPy's ttest_rel also does not output the confidence intervals. The below function calculates the CI using the formula for the paired t-test.

In [111]:
def confint_paired(x1, x2):
    for i in x1, x2:
        x_diff = x1 - x2
        mean_diff = abs(x_diff.mean())
        std_diff = x_diff.std()
        se = x_diff.std() / sqrt(len(x_diff))
        t_val = t.ppf(0.975, (len(x_diff)-1))
        MoE = t_val * se
        confint = mean_diff - MoE, mean_diff + MoE
    print('95 percent confidence interval:', confint)

In [112]:
confint_paired(immer['Y1'], immer['Y2'])

95 percent confidence interval: (6.121953866677087, 25.704712799989572)


### Chi-square Test for Variance

According to Wikipedia, "Cochran's therom shows that variance follows a scaled chi-squared distribution".

Example: A professor is convinced that her students midterm grades have a variance greater than 4. To test her hypothesis, she randomly samples 10 students' midterms results from her class. The grades recorded are the following: 72, 71, 76, 77, 78, 68, 73, 71, 78, 78. Is the professor's claim correct? Test the appropriate hypothesis.

In [157]:
grades = np.array([72, 71, 76, 77, 78, 68, 73, 71, 78, 78])
samplevariance = grades.var(ddof=1)
popvariance = 4
n = len(grades)

# calculate the test statistic: sum of squares about the same means divided by the nominal value for the variance
chi_sq = ((n-1)*samplevariance)/popvariance
chi_sq

29.9

In [162]:
# calculate critical value using chi-squared distribution
critical_val = chi2.ppf(0.95, (n-1))   
p_value = 1 - chi2.cdf(chi_sq, (n-1))

print("Critical value:", crit, "P-value:", p_value)

Critical value: 16.918977604620448 P-value: 0.0004562434471995225


Since the test statistic is greater than the critical value we can reject the null hypothesis that the variance equals to 4. Alternatively because the p-value is less than the alpha value of 0.05, we can reject the null hypothesis. 

## 1.2 Comparing More than Two Groups

In [23]:
from scipy.stats import f_oneway, f

### Sample Design: One-way ANOVA

The ANOVA F-test is used to analyze the difference in population means between two or more groups. With SciPy, f_oneway is used to perform a one-way ANOVA. The following example contains 3 groups (treatments) with 7 observations per group.

In [2]:
group_1 = [18.2, 20.1, 17.6, 16.8, 18.8, 19.7, 19.1]
group_2 = [17.4, 18.7, 19.1, 16.4, 15.9, 18.4, 17.7]
group_3 = [15.2, 18.8, 17.7, 16.5, 15.9, 17.1, 16.7]

In [3]:
# Perform one-way ANOVA on the three treatments. 
f_oneway(group_1,group_2,group_3)

F_onewayResult(statistic=3.968295753691201, pvalue=0.037345340805819825)

The F-statistic is 3.97 and the p-value is 0.037345. We can reject the null hypothesis that the three groups have equal means.

Although we know that there is a signficant difference among the three means, we don't have information on how exactly they differ. We can reject the null that all means are the same but if wish to know which means cause that difference, additional analysis will be required. For that task, we can perform a pairwise t-test to generate the pair-wise comparisons between group means with corrections for multiple testing.

Example 2: Investigate into whether different methods of ventilation during anesthesia has any effect on the red folates.

In [4]:
red_cell_folate = pd.read_csv("red_cell_folate.csv")
red_cell_folate

,folate,ventilation
0,243,"N2O+O2,24h"
1,251,"N2O+O2,24h"
2,275,"N2O+O2,24h"
3,291,"N2O+O2,24h"
4,347,"N2O+O2,24h"
5,354,"N2O+O2,24h"
6,380,"N2O+O2,24h"
7,392,"N2O+O2,24h"
8,206,"N2O+O2,op"
9,210,"N2O+O2,op"


In [5]:
# Extract treatments from dataframe as lists
ventilation_1 = list(red_cell_folate[(red_cell_folate['ventilation']=='N2O+O2,24h')]['folate'])
ventilation_2 = list(red_cell_folate[(red_cell_folate['ventilation']=='N2O+O2,op')]['folate'])
ventilation_3 = list(red_cell_folate[(red_cell_folate['ventilation']=='O2,24h')]['folate'])

# Pass in lists to f_oneway
f_oneway(ventilation_1,ventilation_2,ventilation_3)

F_onewayResult(statistic=3.7113359882669763, pvalue=0.043589334959178244)

The p-value is 0.0436 so we can reject the null hypothesis that the three groups have equal means. 

Note that the f_oneway() function required the sample measurements to be in the form of two or more arrays. However in real-world cases, the treatments are typically found as labels in categorical data (as per the example below). In R, the anova() function allows you to pass in dataframe columns in the form of a linear model formula, e.g. <i>lm(response~treatment)</i>. 

In Python when data is presnted in this form, some preprocessing would be required for compatability with the f_oneway() function. Two different approaches are demonstrated below on how to pass dataframe columns to f_oneway.

Example 3: From the juul dataset, investigate into whether Tanner levels have any significant effect on amount of insulin-like growth factor, igf1. In this problem, The various Tanner levels are the treatments, and the response that we wish to test on is igf1.

In [29]:
juul = pd.read_csv("juul.csv")
print(juul)
# View treatments
print('tanner catergories:', juul['tanner'].unique())

        age  menarche  sex   igf1  tanner  testvol
0       NaN       NaN  NaN   90.0     NaN      NaN
1       NaN       NaN  NaN   88.0     NaN      NaN
2       NaN       NaN  NaN  164.0     NaN      NaN
3       NaN       NaN  NaN  166.0     NaN      NaN
4       NaN       NaN  NaN  131.0     NaN      NaN
5      0.17       NaN  1.0  101.0     1.0      NaN
6      0.17       NaN  1.0   97.0     1.0      NaN
7      0.17       NaN  1.0  106.0     1.0      NaN
8      0.17       NaN  1.0  111.0     1.0      NaN
9      0.17       NaN  1.0   79.0     1.0      NaN
10     0.17       NaN  1.0   43.0     1.0      NaN
11     0.17       NaN  1.0   64.0     1.0      NaN
12     0.25       NaN  1.0   90.0     1.0      NaN
13     0.25       NaN  1.0  141.0     1.0      NaN
14     0.42       NaN  1.0   42.0     1.0      NaN
15     0.50       NaN  1.0   43.0     1.0      NaN
16     0.67       NaN  1.0  132.0     1.0      NaN
17     0.75       NaN  1.0   43.0     1.0      NaN
18     0.75       NaN  1.0   36

The first approach is to iterate through the unique values in the treatment column and for each treatment, select the values from the response column. The list of lists will then expand and can be passed into the f_oneway function as individual arrays.

In [54]:
f_oneway(*[group['igf1'].values for name, group in juul[(juul['igf1'].notnull())].groupby('tanner')])

F_onewayResult(statistic=228.35305434880294, pvalue=4.67821313278659e-130)

Due to the high p-value there is not enough evidence to reject the null hypothesis that there is any difference in igf1 between the various tanner levels. 

The next alternative is to create a dictionary of key-value pairs for treatment and response variables to pass into f_oneway. This is useful if you wish to select and compare only a subset of the treatments.

In [7]:
# Create key-value pairs of response variables for each treatment
tanner = juul['tanner'].unique()
d = {}
for i in tanner:
    d['tanner_%s' % i] = list(juul[(juul['tanner']==i) & (juul['igf1'].notnull())]['igf1'])

# View labels
d.keys()

dict_keys(['tanner_nan', 'tanner_1.0', 'tanner_2.0', 'tanner_3.0', 'tanner_4.0', 'tanner_5.0'])

In [8]:
# Pass lists into f_oneway, we will exclude tanner_nan
f_oneway(d['tanner_1.0'], d['tanner_2.0'], d['tanner_3.0'], d['tanner_4.0'], d['tanner_5.0'])

F_onewayResult(statistic=228.35305434880294, pvalue=4.67821313278659e-130)

<i> Note that the above ANOVA tests assume equal variance. Currently SciPy does not have the option to perform the Welch ANOVA test for unequal variances. This section will be updated with code to perform this task. </i>

### Block Design: Two-way ANOVA

In block design, k treatment means are compared, and blocks are b experiemental units that are similar or homogeneous. One unit within each block is assigned to each treatment. 

Example: The following data frame contains data for nine patients with congestive heart failure before and shortly after administration of enalaprilat. The column 'hr' is the patient heart rate, 'subj' has the subject ids and 'time' indicates time between administration of enalaprilat and measuring the heart rate. Investigate if time or subject has any effect on the heart rates by performing a two way ANOVA. 

The dataframe below indicates that 'subj' are the treatments, 'time' are the blocks, and 'hr' is the response. 

In [35]:
heart_rate = pd.read_csv('heart_rate.csv')
heart_rate

,hr,subj,time
0,96,1,0
1,110,2,0
2,89,3,0
3,95,4,0
4,128,5,0
5,100,6,0
6,72,7,0
7,79,8,0
8,100,9,0
9,92,1,30


Currently SciPy has no feature to compute a two way ANOVA. Without the help of other packages like statsmodels you will need to code it yourself to perform this task. Below is function based on the randomized block design formula. The inputs are the column names for the response, treatments and blocks, followed by data. The output is the ANOVA table similar to the output in R with the anova() function.

In [36]:
def f_twoway(values, treatments, blocks, data):
    
    # Compute correction of the means:
    totals = []
    for i in data[treatments].unique():
        totals.append(data[(data[treatments]==i)][values].sum())
        G = sum(totals)
        CM = (G**2)/(data[blocks].nunique()*data[treatments].nunique())    
    
    # Compute sum of squares:
    ss = []
    for i in data[values]:
        ss.append(i**2)
        total_SS = sum(ss) - CM    
    ss_t = []
    for i in data[treatments].unique():
        ss_t.append(data[(data[treatments]==i)][values].sum())
        SST = (sum([i**2 for i in ss_t]))/(data[blocks].nunique())-CM
    ss_b = []
    for i in data[blocks].unique():
        ss_b.append(data[(data[blocks]==i)][values].sum())
        SSB = (sum([i**2 for i in ss_b]))/(data[treatments].nunique())-CM
    SSE = total_SS - SST - SSB    
    
    # Compute df and mean sq:
    df_treatment = data[treatments].nunique()-1
    df_block = data[blocks].nunique()-1
    df_error = df_treatment*df_block
    MST = SST/df_treatment
    MSB = SSB/df_block
    MSE = SSE/df_error
    
    # test statistics and p-values:
    F_treatment = MST/MSE 
    F_block = MSB/MSE
    p_treatment = f.sf(F_treatment, df_treatment, df_error)
    p_block = f.sf(F_block, df_block, df_error)
    
    # Build the ANOVA table similar to R
    anova_table = pd.DataFrame(index=[treatments, blocks, 'Residuals'])
    anova_table['Df'] = [df_treatment, df_block, df_error]
    anova_table['Sum Sq'] = [SST, SSB, SSE]
    anova_table['Mean Sq'] = [MST, MSB, MSE]
    anova_table['F Value'] = [F_treatment, F_block, '']
    anova_table['Pr(>F)'] = [p_treatment, p_block, '']
    return(anova_table)

# To do: update function to handle NaN values in columns. 

In [37]:
f_twoway('hr', 'subj', 'time', heart_rate)

,Df,Sum Sq,Mean Sq,F Value,Pr(>F)
subj,8,8966.555556,1120.819444,90.6391,4.86268e-16
time,3,150.972222,50.324074,4.06964,0.0180205
Residuals,24,296.777778,12.365741,,


The above is identical to the output given by R's anova() function. From the low p-values, we can reject the null hypothesis that the mean heartrate is the same between subjects and time.

# 2. Non-Parametric (Non-Normal Distribution) Testing

Non-parametric tests do not make any assumptions with respect to the underlying distribution of the data, i.e. it does not assume data is normally distributed or have the same variance. The data cannot be measured quantitatively therefore measurements will be done through ranking. 

## 2.1 Comparing Two Groups

In [40]:
from scipy.stats import ranksums, wilcoxon, mannwhitneyu, rankdata

### Wilcoxon Rank Sum Test

The Wilcoxon Rank Sum Test is the rank equivalent of the unpaired t-test. In SciPy, the ranksums function computes the Wilcoxon rank-sum statistic for two samples. However ranksums is unable to handle ties in ranks between two samples. In this case, the mannwhitneyu function is more appropriate. 

Example: A data science student is using two machine learning algorithms to analyze a given dataset. First she tests her model with a Decision Tree using a 10-fold Cross Validation and records the Accuracy for each fold. She then repeats using a 12-fold Logistic Regression algorithm on the same dataset. The accuracy for each fold of the algorithms are given as follows:

In [42]:
DT = [75, 79, 69, 78, 65, 87, 74, 81, 77, 88]
LR = [85, 68, 78, 73, 69, 76, 69, 80, 73, 67, 78, 82]

In [48]:
mannwhitneyu(DT, LR, alternative='two-sided')

MannwhitneyuResult(statistic=73.0, pvalue=0.40861810325767045)

Since p-value > 0.05 we do not reject the null hypothesis that both algorithms perform similarily. 73 is the the statistic for the Mann Whitney U Test (denoted by U). According to R documentation for its wilcox.test, the statistic computed for the rank sum test is also the U statistic, despite having the label 'W'. 

To calculate the W statistic manually:

In [49]:
def sum_rank_statistic (x, y):
    
    n = ['n1' for i in range(len(x))] + ['n2' for i in range(len(y))]
    rank = list(rankdata(x + y, method='average')) # rank all observations and average the tied elements
    
    if x < y:  # The smaller sample size is used for 'n1'
        T1 = sum([rank for n, rank in zip(n, rank) if n=='n1']) # sum the ranks for n1
        T1_ = len(x)*(len(x + y)+1) - T1 # calcualte n1(n1+n2+1)-T1
    else:
        T1 = sum([rank for n, rank in zip(n, rank) if n=='n2']) 
        T1_ = len(y)*(len(x + y)+1) - T1
    
    T = min(T1, T1_) # min of T1 and T1*
    
    print("T1 is", T1)
    print('T1* is', T1_)
    print('T is', T)
    
sum_rank_statistic(DT, LR)

T1 is 128.0
T1* is 102.0
T is 102.0


T1 is the test statistic for a left-tailed test, T1* is used for a right-tailed test, and T= min(T1,T1*) for a two-tailed test. 

### Wilcoxon Signed-Rank Test

The Wilcoxon signed-rank test is the rank equivalent of the paired t-test and is used when the data is non-parametric and both samples are paired. The wilcoxon() function from SciPy is used to conduct the signed-rank test.

Example: The manager of a park wants to see if pollution levels in the lake are reduced when boats are not allowed. This is measured by the rate of pollution every 60 minutes for a day when boats are allowed, and a seperate day when they are not. Below are the measurements:

In [50]:
a = [159, 135, 141, 101, 102, 168, 62, 167, 174, 159, 66, 118, 181, 171, 112]
b = [214, 159, 169, 202, 103, 119, 200, 109, 132, 142, 194, 104, 219, 119, 234]

In [51]:
wilcoxon(a, b, correction=True)

WilcoxonResult(statistic=40.0, pvalue=0.2680667604057142)

There is not enough evidence to reject the null hypothesis that the populations levels are different between both days.

Note that the documentation for the wilcoxon() states, <i> "Use only when the number of observation in each sample is > 20". </i> This is because SciPy uses a normal approximation when calculating the p-value which works for large samples sizes, however the p-value can differ from the exact p-value for smaller samples like in this example. In contrast R's wilcox.test() function will always calculate the exact P-value unless specified for samples less than 50. 

To calculate the test statistics manually:

In [130]:
def signed_rank_statistic(x, y):

    d = [x - y for x, y in zip(x, y)] # Find the difference in the values in both lists
    rank = rankdata([abs(i) for i in d], method='average') # rank the absolute values of the differences

    T_pos = sum([rank for d, rank in zip(d, rank) if d > 0]) # sum all ranks where differences are postive
    T_neg = sum([rank for d, rank in zip(d, rank) if d < 0]) # " " " " negative
    T = min(T_pos, T_neg) # min of T_pos and T_neg

    print("T+ is", T_pos)
    print('T- is', T_neg)
    print('T is', T)
    
signed_rank_statistic(Zip, Tar)

T+ is 40.0
T- is 80.0
T is 40.0


T- is the test statistic for a one-tailed test, T=min(T+,T-) is the test statistic for a two-tailed test. 

## 2.2 Comparing More than Two Groups

In [52]:
from scipy.stats import kruskal, friedmanchisquare

### Kruskal-Wallis Test 

The Kruskal-Wallis H test is the non-parametric, rank-based equivalent of the one-way ANOVA test. It can be used to determine if there are statistically significant differences between two or more groups of an independent variable on a continuous or ordinal dependent variable. The hypotheses for the test are: H0: population medians are equal and H1: population medians are not equal. The kruskal() function from SciPy is used to perform the Kruskal-Wallis test. 

Like the f_oneway(), the kruskal() function (and friedmanchisquare() below) takes in two or more arrays as the sample measurements to be compared. In R, the kruskal.test() function allows you to pass in dataframe columns as a formula, e.g. <i>kruskal.test(response~group)</i>, which is more applicable to real-world applications. To preprocess the data for compatibility with the kruskal() function, we use the same two approaches from the f_oneway() example.

Example: Suppose data for the wages between three groups are given in the following form and we would like to determine if there is any difference in the mean wages between the three groups:

In [56]:
salary = [45, 55, 60, 70, 72, 23, 41, 54, 66, 90, 20, 30, 34, 40, 44]
group = ['men' for i in range(5)]+['women' for i in range(5)]+['minority' for i in range(5)]
wages = pd.DataFrame({'salary': salary, 'group': group})
wages

,salary,group
0,45,men
1,55,men
2,60,men
3,70,men
4,72,men
5,23,women
6,41,women
7,54,women
8,66,women
9,90,women


In [57]:
kruskal(*[group['salary'].values for name, group in wages.groupby('group')])

KruskalResult(statistic=6.720000000000006, pvalue=0.03473525894473845)

In [15]:
d = {}
for i in wages['group'].unique():
    d['group_%s' % i] = list(wages[(wages['group']==i)]['salary'])
    
kruskal(d['group_men'],d['group_women'],d['group_minority'])

KruskalResult(statistic=6.720000000000006, pvalue=0.03473525894473845)

### Friedman Test

The Friedman test is the rank equivalent of the randomized block design with k treatments and b blocks. All K measuresments within a block are ranked. Sums of the ranks of the k observations are then used to compare the k treatment distributions. The SciPy function for the Friedman test is friedmanchisquare().


Example: A student is using three machine learning algorithms to analyze a given dataset. She tests her model using the Decision Tree, Logistic Regression and Naive Bayes algorithm. She uses 10-fold Cross Validation by dividing her dataset into 10 parts. Nine of those parts are use for training and one tenth for testing. She repeats this procedure 10 times - each time reserving a different tenth for testing. The accuracy for each fold of the algorithms are given as follows:

In [60]:
DT = [75, 79, 69, 78, 65, 87, 74, 81, 77, 85]
LR = [85, 68, 78, 73, 69, 76, 69, 80, 73, 67]
NB = [86, 75, 79, 82, 68, 69, 77, 81, 80, 79]
testnames = np.repeat(['DT','LR','NB'], 10)
folds = list(range(1,11))*3
TestModel = pd.DataFrame({'Accuracy': DT+LR+NB, 'Testnames': testnames, 'Folds': folds})
TestModel

,Accuracy,Testnames,Folds
0,75,DT,1
1,79,DT,2
2,69,DT,3
3,78,DT,4
4,65,DT,5
5,87,DT,6
6,74,DT,7
7,81,DT,8
8,77,DT,9
9,85,DT,10


In [61]:
friedmanchisquare(*[group['Accuracy'].values for name, group in TestModel.groupby('Testnames')])

FriedmanchisquareResult(statistic=4.6666666666666785, pvalue=0.09697196786440448)

In [62]:
d = {}
for i in TestModel['Testnames'].unique():
    d[i] = list(TestModel[(TestModel['Testnames']==i)]['Accuracy'])

friedmanchisquare(d['DT'],d['LR'],d['NB'])

FriedmanchisquareResult(statistic=4.6666666666666785, pvalue=0.09697196786440448)

p-value is >0.05 hence we do not reject the Null Hypothesis. 
We do not have enough evidence to suggest that any ML algorithm is better in accuracy compared to the others.